<a href="https://colab.research.google.com/github/sihan827/2021_Winter_PE/blob/main/evaluation_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Evaluation Metric 분석

# 아래 변수들은 모델 성능을 평가하기 위한 클래스 AverageMeter의 인스턴스
# 순서대로 top-1, top-5, 손실 관련 평가 지표를 저장하는 데 사용
top1_meter = AverageMeter()
top5_meter = AverageMeter()
loss_meter = AverageMeter()

# 테스트 함수
def test(loader):
    # 모델을 eval 모드로 바꾸어 모델을 테스트하는 동안 dropout 등으로 인해
    # 신경망의 일부를 사용하지 못하게 하는 것을 방지
    model.eval()
    # 기본적으로 torch는 autograd=True로 명시된 파라미터들이 기록에 남아서 
    # 나중에 역전파 알고리즘 수행시 이 기록을 참고하여 미분을 하여 수행함
    # 테스트 시에는 각 파라미터들을 최적화하지 않으므로 미분값을 계산할 필요가 없고
    # 이는 torch.no_grad를 통해 파라미터들이 기록에 남지 않아서 메모리를 아낄 수 있음
    with torch.no_grad():
        # 학습과정과 유사하게 enumerate함수와 반복문을 통해 
        # 한 배치의 인덱스, 데이터, 라벨을 불러옴
        for step, (X, y) in enumerate(loader):
            # cuda 사용설정 
            X, y = X.cuda(), y.cuda()
            # 한 배치의 크기를 N에 저장
            N = X.shape[0]

            # 모델을 통한 예측값 계산
            outs = model(X)
            # 지정한 손실함수를 통해 예측값과 실제 라벨값 사이의 손실값 계산
            loss = criterion(outs, y)
            # 미리 지정된 accuracy 함수를 통해 현재 배치에 대하여
            # 예측값과 실제 라벨 사이 top-1, top-5 정답률을 계산
            prec1, prec3 = accuracy(outs, y, topk=(1,5))
            # 처음에 선언한 AverageMeter의 각 인스턴스 안의 지표들을 업데이트
            # (총합, 평균, 개수)
            top1_meter.update(prec1.item(), N)
            top5_meter.update(prec3.item(), N)
            loss_meter.update(loss.item(), N)

    # 각 인스턴스에 저장된 평균값을 가져옴
    # top-1, top-5에서 평균값은 정답률을 의미하고
    # loss에서 평균값은 말그대로 해당 배치 데이터들의 평균값을 의미 
    top1_avg = top1_meter.get_avg()
    top5_avg = top5_meter.get_avg()
    loss_avg = loss_meter.get_avg()

    # 한 배치에 대하여 각 평가 파라미터들을 계산했으므로
    # 해당 클래스의 reset 함수로 모든 값을 초기화
    for avg in [top1_meter, top5_meter, loss_meter]:
        avg.reset()

    # 평가지표 출력
    print('[Test] Loss: {loss:.4f}, Top1: {top1: .4f}, Top5: {top5: .4f}'.format(
        loss=loss_avg,
        top1=top1_avg,
        top5=top5_avg
    ))